In [10]:
import cv2
import numpy as np
from pathlib import Path
import time
import psutil
import os
import gc
import math

In [11]:
class Panorama:
    def __init__(self, initial_frame):
        """初始化全景图系统"""
        self.frame_h, self.frame_w = initial_frame.shape[:2]
        
        # 创建初始画布（给一些边距以便扩展）
        margin = 100
        self.canvas = np.zeros((self.frame_h + 2*margin, self.frame_w + 2*margin, 3), dtype=np.uint8)
        
        # 当前位置（从中心开始）
        self.current_x = margin
        self.current_y = margin
        
        # 记录已使用区域的边界
        self.min_x = margin
        self.max_x = margin + self.frame_w
        self.min_y = margin
        self.max_y = margin + self.frame_h
        
        # 放置第一帧
        self.canvas[margin:margin+self.frame_h, margin:margin+self.frame_w] = initial_frame

    def create_alpha_mask(self, dx, dy):
        """创建带有最小混合区域的alpha遮罩"""
        # 设置最小混合宽度为图像尺寸的10%
        min_blend_width = min(self.frame_w, self.frame_h) // 10
        
        alpha = np.zeros((self.frame_h, self.frame_w), dtype=np.float32)
        
        if abs(dx) > abs(dy):  # 主要是水平移动
            blend_width = max(int(abs(dx)), min_blend_width)  # 确保最小宽度
            if dx > 0:  # 向右移动
                alpha[:, :blend_width] = np.linspace(0, 1, blend_width)
                alpha[:, blend_width:] = 1
            else:  # 向左移动
                alpha[:, -blend_width:] = np.linspace(1, 0, blend_width)
                alpha[:, :-blend_width] = 1
        else:  # 主要是垂直移动
            blend_width = max(int(abs(dy)), min_blend_width)  # 确保最小宽度
            if dy > 0:  # 向下移动
                alpha[:blend_width, :] = np.linspace(0, 1, blend_width)[:, np.newaxis]
                alpha[blend_width:, :] = 1
            else:  # 向上移动
                alpha[-blend_width:, :] = np.linspace(1, 0, blend_width)[:, np.newaxis]
                alpha[:-blend_width, :] = 1
                
        return np.stack([alpha] * 3, axis=2)
    
    def expand_canvas_if_needed(self, new_x, new_y):
        """在需要时扩展画布"""
        need_expand = False
        pad_left = pad_right = pad_top = pad_bottom = 0
        
        # 检查是否需要扩展
        if new_x < 0:
            pad_left = abs(new_x)
            need_expand = True
        if new_x + self.frame_w > self.canvas.shape[1]:
            pad_right = new_x + self.frame_w - self.canvas.shape[1]
            need_expand = True
        if new_y < 0:
            pad_top = abs(new_y)
            need_expand = True
        if new_y + self.frame_h > self.canvas.shape[0]:
            pad_bottom = new_y + self.frame_h - self.canvas.shape[0]
            need_expand = True
            
        if need_expand:
            # 创建新画布
            new_h = self.canvas.shape[0] + pad_top + pad_bottom
            new_w = self.canvas.shape[1] + pad_left + pad_right
            new_canvas = np.zeros((new_h, new_w, 3), dtype=np.uint8)
            
            # 复制原画布内容到新位置
            y_start = pad_top
            x_start = pad_left
            new_canvas[y_start:y_start+self.canvas.shape[0], 
                      x_start:x_start+self.canvas.shape[1]] = self.canvas
            
            # 更新坐标
            self.current_x += pad_left
            self.current_y += pad_top
            self.min_x += pad_left
            self.max_x += pad_left
            self.min_y += pad_top
            self.max_y += pad_top
            
            self.canvas = new_canvas
            return pad_left, pad_top
            
        return 0, 0
    
    def add_frame(self, frame, angle, magnitude):
        """添加新帧到全景图"""
        # 计算新位置
        dx = magnitude * math.cos(math.radians(angle + 90))
        dy = magnitude * math.sin(math.radians(angle - 90))
        
        new_x = int(self.current_x + dx)
        new_y = int(self.current_y + dy)
        
        # 如果需要则扩展画布
        offset_x, offset_y = self.expand_canvas_if_needed(new_x, new_y)
        new_x += offset_x
        new_y += offset_y
        
        # 计算重叠区域
        roi = self.canvas[new_y:new_y+self.frame_h, new_x:new_x+self.frame_w]
        
        # 创建alpha遮罩
        alpha = self.create_alpha_mask(dx, dy)
        
        # 根据垂直移动方向决定叠放顺序
        is_moving_down = dy > 0 and abs(dy) > abs(dx)
        if is_moving_down:
            # 向下移动时，全景图叠加在新帧上面
            result = alpha * roi + (1 - alpha) * frame
        else:
            # 其他情况，新帧叠加在全景图上面
            result = (1 - alpha) * roi + alpha * frame
        
        # 更新画布
        self.canvas[new_y:new_y+self.frame_h, new_x:new_x+self.frame_w] = result.astype(np.uint8)
        
        # 更新位置信息
        self.current_x = new_x
        self.current_y = new_y
        self.min_x = min(self.min_x, new_x)
        self.max_x = max(self.max_x, new_x + self.frame_w)
        self.min_y = min(self.min_y, new_y)
        self.max_y = max(self.max_y, new_y + self.frame_h)
    
    def get_result(self):
        """获取最终结果"""
        return self.canvas[self.min_y:self.max_y, self.min_x:self.max_x]


In [12]:
def calculate_movement(img1, img2):
    """计算两帧之间的运动方向和幅度"""
    # 获取图像尺寸用于调试信息
    h, w = img1.shape[:2]
    
    sift = cv2.SIFT_create()
    keypoints1, descriptors1 = sift.detectAndCompute(img1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(img2, None)
    
    print(f"\n== 调试信息 ==")
    print(f"图像尺寸: {w}x{h}")
    print(f"检测到的特征点数量: 帧1={len(keypoints1)}, 帧2={len(keypoints2)}")
    
    if descriptors1 is None or descriptors2 is None:
        print("未检测到特征点")
        return None, None
    
    # 特征匹配
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)
    
    # 收集特征点位置信息用于分析分布
    frame1_points = []
    frame2_points = []
    
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)
            # 记录匹配点的坐标
            pt1 = keypoints1[m.queryIdx].pt
            pt2 = keypoints2[m.trainIdx].pt
            frame1_points.append(pt1)
            frame2_points.append(pt2)
    
    print(f"良好匹配点数量: {len(good_matches)}")
            
    if len(good_matches) < 10:
        print("匹配点数量不足")
        return None, None
        
    # 计算特征点分布
    frame1_points = np.array(frame1_points)
    frame2_points = np.array(frame2_points)
    
    # 计算特征点的分布统计
    if len(frame1_points) > 0:
        x_min = np.min(frame1_points[:, 0])
        x_max = np.max(frame1_points[:, 0])
        y_min = np.min(frame1_points[:, 1])
        y_max = np.max(frame1_points[:, 1])
        print(f"特征点分布范围: X=({x_min:.1f}, {x_max:.1f}), Y=({y_min:.1f}, {y_max:.1f})")
    
    # 计算平均移动向量
    movements = []
    for match in good_matches:
        pt1 = np.array(keypoints1[match.queryIdx].pt)
        pt2 = np.array(keypoints2[match.trainIdx].pt)
        movement = pt2 - pt1
        movements.append(movement)
    
    mean_movement = np.mean(movements, axis=0)
    dx, dy = mean_movement
    
    # 计算移动统计信息
    movements = np.array(movements)
    std_dev = np.std(movements, axis=0)
    print(f"平均移动: dx={dx:.2f}, dy={dy:.2f}")
    print(f"移动标准差: dx_std={std_dev[0]:.2f}, dy_std={std_dev[1]:.2f}")
    
    # 计算角度（0度为正上方，顺时针旋转）
    angle = math.degrees(math.atan2(-dy, dx))  # 使用-dy是因为图像坐标系y轴向下
    angle = (angle + 90) % 360
    
    # 计算移动幅度
    magnitude = math.sqrt(dx*dx + dy*dy)
    
    # 判断主要移动方向并调整幅度
    is_horizontal = abs(dx) > abs(dy)
    if is_horizontal:
        magnitude = magnitude * 1.5  # 横向移动时增加magnitude以匹配纵向效果
    
    # 输出移动方向相关信息
    move_direction = "横向" if is_horizontal else "纵向"
    print(f"主要移动方向: {move_direction}")
    print(f"移动角度: {angle:.2f}°")
    print(f"移动幅度: {magnitude:.2f}")
    print("================\n")
    
    return angle, magnitude



In [13]:
def get_memory_usage():
    """获取当前进程的内存使用量（MB）"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024


In [14]:
def process_video(video_path, start_frame=0, frame_interval=1):
    """处理视频"""
    # 创建输出目录
    Path('output').mkdir(exist_ok=True)
    
    # 设置最大显示尺寸
    MAX_DISPLAY_WIDTH = 1920
    MAX_DISPLAY_HEIGHT = 1080
    
    def resize_to_screen(image):
        """调整图像大小以适应屏幕，保持宽高比"""
        height, width = image.shape[:2]
        
        # 计算缩放比例
        width_ratio = MAX_DISPLAY_WIDTH / width
        height_ratio = MAX_DISPLAY_HEIGHT / height
        scale = min(width_ratio, height_ratio, 1.0)  # 不放大，只缩小
        
        if scale < 1.0:
            new_width = int(width * scale)
            new_height = int(height * scale)
            resized = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)
            return resized, new_width, new_height
        return image, width, height
    
    # 初始化性能监控变量
    start_time = time.time()
    max_memory = 0
    memory_readings = []
    
    # 创建窗口
    cv2.namedWindow('Panorama Progress', cv2.WINDOW_NORMAL)  # 改为WINDOW_NORMAL
    
    # 打开视频
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("无法打开视频文件")
    
    try:
        # 跳到起始帧
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        
        # 读取第一帧
        ret, prev_frame = cap.read()
        if not ret:
            raise ValueError("无法读取第一帧")
            
        # 初始化全景图
        panorama = Panorama(prev_frame)
        frame_count = 1
        
        # 显示初始状态
        initial_result = panorama.get_result()
        display_result, win_width, win_height = resize_to_screen(initial_result)
        cv2.resizeWindow('Panorama Progress', win_width, win_height)  # 设置窗口大小
        cv2.imshow('Panorama Progress', display_result)
        cv2.waitKey(1)
        
        while True:
            # 检查是否按下 'q' 键退出
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                print("\n用户终止处理")
                cv2.destroyAllWindows()
                return
            
            # 跳过指定数量的帧
            for _ in range(frame_interval - 1):
                ret = cap.grab()
                if not ret:
                    break
            
            # 读取当前帧
            ret, curr_frame = cap.read()
            if not ret:
                break
                
            # 计算运动
            angle, magnitude = calculate_movement(prev_frame, curr_frame)
            
            # 检查计算结果是否有效
            if angle is None or magnitude is None:
                print("\n无法计算帧间运动，跳过当前帧")
                continue
                
            # 添加到全景图
            try:
                panorama.add_frame(curr_frame, angle, magnitude)
                print(f"\r处理第 {frame_count} 帧 - 方向: {angle:.1f}°, 幅度: {magnitude:.1f}", end="")
                
                # 获取当前结果并显示
                current_result = panorama.get_result()
                display_result, win_width, win_height = resize_to_screen(current_result)
                cv2.resizeWindow('Panorama Progress', win_width, win_height)  # 更新窗口大小
                cv2.imshow('Panorama Progress', display_result)
                
            except Exception as e:
                print(f"\n处理帧时出错: {str(e)}")
                break
            
            # 监控内存使用
            current_memory = get_memory_usage()
            memory_readings.append(current_memory)
            max_memory = max(max_memory, current_memory)
            
            prev_frame = curr_frame.copy()
            frame_count += 1
            
            # 定期清理内存
            if frame_count % 100 == 0:
                gc.collect()
            
        # 保存结果
        result = panorama.get_result()
        cv2.imwrite('output/panorama.jpg', result)
        print(f"\n处理完成，共处理 {frame_count} 帧")
        
        # 显示最终结果
        cv2.namedWindow('Final Panorama', cv2.WINDOW_NORMAL)
        final_display, win_width, win_height = resize_to_screen(result)
        cv2.resizeWindow('Final Panorama', win_width, win_height)
        cv2.imshow('Final Panorama', final_display)
        print("\n按任意键关闭窗口...")
        cv2.waitKey(0)
        
        # 计算并显示性能统计
        end_time = time.time()
        total_time = end_time - start_time
        avg_memory = sum(memory_readings) / len(memory_readings)

        # 性能统计
        print(f"\n\n=== 性能统计 ===")
        print(f"总处理时间: {total_time:.2f} 秒")
        print(f"平均每帧处理时间: {total_time/frame_count:.2f} 秒")
        print(f"最大内存使用: {max_memory:.2f} MB")
        print(f"平均内存使用: {avg_memory:.2f} MB")
        print(f"总处理帧数: {frame_count}")
        print("================")
        
    finally:
        cap.release()
        gc.collect()


In [17]:
if __name__ == "__main__":
    video_path = "video/gtav/3.mp4"  # 替换为实际的视频路径
    process_video(video_path, start_frame=1, frame_interval=10)


== 调试信息 ==
图像尺寸: 1920x1080
检测到的特征点数量: 帧1=13418, 帧2=13026
良好匹配点数量: 7122
特征点分布范围: X=(4.0, 1914.5), Y=(5.6, 1037.3)
平均移动: dx=0.06, dy=38.38
移动标准差: dx_std=18.85, dy_std=19.37
主要移动方向: 纵向
移动角度: 0.08°
移动幅度: 38.38

处理第 1 帧 - 方向: 0.1°, 幅度: 38.4
== 调试信息 ==
图像尺寸: 1920x1080
检测到的特征点数量: 帧1=13026, 帧2=12571
良好匹配点数量: 7356
特征点分布范围: X=(2.9, 1915.3), Y=(2.7, 1052.1)
平均移动: dx=-0.34, dy=37.09
移动标准差: dx_std=37.27, dy_std=25.06
主要移动方向: 纵向
移动角度: 359.47°
移动幅度: 37.09

处理第 2 帧 - 方向: 359.5°, 幅度: 37.1
== 调试信息 ==
图像尺寸: 1920x1080
检测到的特征点数量: 帧1=12571, 帧2=12005
良好匹配点数量: 6910
特征点分布范围: X=(2.5, 1916.1), Y=(5.2, 1062.2)
平均移动: dx=-0.06, dy=35.92
移动标准差: dx_std=28.41, dy_std=16.76
主要移动方向: 纵向
移动角度: 359.90°
移动幅度: 35.92

处理第 3 帧 - 方向: 359.9°, 幅度: 35.9
== 调试信息 ==
图像尺寸: 1920x1080
检测到的特征点数量: 帧1=12005, 帧2=11684
良好匹配点数量: 7001
特征点分布范围: X=(4.3, 1916.5), Y=(4.5, 1075.6)
平均移动: dx=-0.25, dy=34.86
移动标准差: dx_std=25.00, dy_std=16.76
主要移动方向: 纵向
移动角度: 359.59°
移动幅度: 34.86

处理第 4 帧 - 方向: 359.6°, 幅度: 34.9
== 调试信息 ==
图像尺寸: 1920x1080
检测到的特征点数量: 帧1

In [16]:
if __name__ == "__main__":
    video_path = "video/drone/3.mp4"  # 替换为实际的视频路径
    process_video(video_path, start_frame=1, frame_interval=10)


== 调试信息 ==
图像尺寸: 1920x1080
检测到的特征点数量: 帧1=8571, 帧2=9359
良好匹配点数量: 1446
特征点分布范围: X=(8.3, 1911.4), Y=(9.0, 1066.8)
平均移动: dx=9.08, dy=61.47
移动标准差: dx_std=74.66, dy_std=78.69
主要移动方向: 纵向
移动角度: 8.40°
移动幅度: 62.14

处理第 1 帧 - 方向: 8.4°, 幅度: 62.1
== 调试信息 ==
图像尺寸: 1920x1080
检测到的特征点数量: 帧1=9359, 帧2=9997
良好匹配点数量: 973
特征点分布范围: X=(3.8, 1916.7), Y=(12.2, 996.2)
平均移动: dx=12.39, dy=90.75
移动标准差: dx_std=105.36, dy_std=76.16
主要移动方向: 纵向
移动角度: 7.78°
移动幅度: 91.59

处理第 2 帧 - 方向: 7.8°, 幅度: 91.6
== 调试信息 ==
图像尺寸: 1920x1080
检测到的特征点数量: 帧1=9997, 帧2=10637
良好匹配点数量: 1510
特征点分布范围: X=(12.2, 1897.5), Y=(12.4, 1065.6)
平均移动: dx=14.67, dy=67.24
移动标准差: dx_std=82.32, dy_std=57.64
主要移动方向: 纵向
移动角度: 12.31°
移动幅度: 68.83

处理第 3 帧 - 方向: 12.3°, 幅度: 68.8
== 调试信息 ==
图像尺寸: 1920x1080
检测到的特征点数量: 帧1=10637, 帧2=11087
良好匹配点数量: 888
特征点分布范围: X=(7.9, 1904.9), Y=(12.2, 1065.8)
平均移动: dx=12.75, dy=88.79
移动标准差: dx_std=129.07, dy_std=81.07
主要移动方向: 纵向
移动角度: 8.17°
移动幅度: 89.70

处理第 4 帧 - 方向: 8.2°, 幅度: 89.7
== 调试信息 ==
图像尺寸: 1920x1080
检测到的特征点数量: 帧1=11087, 帧2=1